UNIVERSIDADE FEDERAL DE PELOTAS

CDTEC - ENG. DE COMPUTAÇÃO - KATHE BESERRA 

Relatório 3 – Interpolação e Ajuste de Função

In [ ]:
import math
import numpy as np
from numpy import *
from sympy import *
import matplotlib.pyplot as plt

In [ ]:
def metNewton(vet_X,fx_Y,degree,value,p):
  i=0
  p_n=0
  sum=1
  x = Symbol('x') 

  #matriz onde é salvo os valores de d
  d = np.zeros((len(vet_X),degree))


  if degree < len(vet_X):
    while i < degree + 1:
      if i == 0:
        d[0,i] = fx_Y[i]
        p_n = fx_Y[i]
      else:
        #adiciona 1 em j cada vez que entro em uma nova ordem 
        j = i-1

        #realiza os calculos do valor d até a ordem desejada
        while j < degree:
          if i == 1:
            d[i,j] = (fx_Y[j+1] - fx_Y[j])/(vet_X[j+1]-vet_X[j])
          elif i != 0:
            d[i,j] = (d[i-1,j] - d[i-1,j-1])/(vet_X[i]-vet_X[0])
          j = j + 1
      #calcula p
      if i >= 1:
        sum = sum * (x-vet_X[i-1])
        p_n = p_n + d[i,i-1]*sum
      i = i + 1

    print("\nP",degree,"(x) = ", p_n)
    print("O valor P_n(", value, ") =", round(p_n.subs(x,value),p))
    return round(p_n.subs(x,value),p)
  else:
    print("\nPolinomio de grau ", degree, "nao pode ser alcancado")

In [ ]:
def metLagrange(vet_X, fx_Y, value, p):
  n=0
  p_n=0
  pn_print=""
  x = Symbol('x')
  for i in vet_X:
    t_mult = 1
    tdiv_mult = 1
    
    for j in vet_X: 
      #Soma valores para divisao Lk
      if i != j:
        t_mult = t_mult * (x-j)
        tdiv_mult = tdiv_mult * (i-j)


    
    # Soma P_n = Yi*Lk
    p_n = p_n + fx_Y[n]*t_mult/tdiv_mult
    n=n+1
  
  print("Pn(x): ", p_n)
  print("Valor interpolado para ", value, ": ", round(p_n.subs(x,value),5))

In [ ]:
# vet_x -> vetor x
# fx_y -> vetor y
# value -> valor para calculo de sk

def metSplineCub(vet_x, fx_y, value):
  h = np.zeros(len(vet_x))
  l = np.zeros(len(vet_x))
  u = np.zeros(len(vet_x))
  z = np.zeros(len(vet_x))
  c = np.zeros(len(vet_x))
  b = np.zeros(len(vet_x))
  d = np.zeros(len(vet_x))
  det = np.zeros(len(vet_x))

  #define espaçamentos em X
  for i in range(len(vet_x)-1):
    h[i] = vet_x[i+1] - vet_x[i]

  # Calculo termo independente
  for i in range(1, len(vet_x)-1):
    det[i] = ((3/h[i])*(fx_y[i+1]-fx_y[i])) - (3/h[i-1])*(fx_y[i]-fx_y[i-1])

  l[0] = 1
  u[0] = 0
  z[0] = 0
  for i in range(1, len(vet_x)-1):
    l[i] = ((2*(vet_x[i+1]-vet_x[i-1])) - h[i-1]*u[i-1])
    u[i] = h[i]/l[i]
    z[i] = (det[i]-h[i-1]*z[i-1])/l[i]
  
  l[len(l)-1] = 1
  z[len(z)-1] = 0
  c[len(c)-1] = 0

  #resolve sistema
  i = len(vet_x)-2
  while i >= 0:
    c[i] = z[i]-(u[i]*c[i+1])
    b[i] = (fx_y[i+1] - fx_y[i])/h[i] - h[i]*(c[i+1] + 2*c[i])/3
    d[i] = (c[i+1]-c[i])/(3*h[i])
    i = i - 1

  #print("Valores de ak: ", fx_y)
  #print("\nValores de bk: ", b)
  #print("\nValores de ck: ", c)
  #print("\nValores de dk: ", d)
  i=0
  while i < len(vet_x):
    if (value > vet_x[i]) and (value < vet_x[i+1]):
      print("S(x)=", round(fx_y[i],5),"+",round(b[i],5),"*(x-",vet_x[i],") +",c[i],"*(x-",vet_x[i],")2 + ", round(d[i],5),"*(x-",vet_x[i],")3")
      sk = fx_y[i] + b[i]*(value-vet_x[i]) + c[i]*math.pow((value-vet_x[i]),2) + d[i]*math.pow((value-vet_x[i]),3)
      print("S(",value,")= ",round(sk,5))
      return
    elif value == vet_x[i]:
      return print("S(",value,")= ", fx_y[value])
    else:
      i = i + 1


In [ ]:
# AJUSTES DE FUNÇÕES

# LINEAR
def metMMQ(vet_x, fx_y):
  #SOMATORIO
  sumXY=0; sumX=0; sumY=0; sumX2=0
  for i in range(len(vet_x)):
    sumXY = sumXY + vet_x[i] * fx_y[i]
    sumX = sumX + vet_x[i]
    sumY = sumY + fx_y[i] 
    sumX2 = sumX2 + math.pow(vet_x[i],2)

  a = ((len(vet_x)*sumXY)-sumX*sumY)/(len(vet_x)*sumX2 - math.pow(sumX,2))
  b = ((sumX*sumXY) - sumY*sumX2)/(math.pow(sumX,2) - len(vet_x)*sumX2)

  f_x = lambda x: a*x + b

  return f_x

In [ ]:
# a*b^x
def metMMQABX(vet_x,fx_y):
  sumXY=0; sumX=0; sumY=0; sumX2=0
  for i in range(len(vet_x)):
    sumXY = sumXY + np.log(vet_x[i]) * np.log(fx_y[i])
    sumX = sumX + np.log(vet_x[i])
    sumY = sumY + np.log(fx_y[i])
    sumX2 = sumX2 + math.pow(np.log(vet_x[i]),2)


  b = ((len(vet_x)*sumXY)-sumX*sumY)/(len(vet_x)*sumX2 - math.pow(sumX,2))
  a = ((sumX*sumXY) - sumY*sumX2)/(math.pow(sumX,2) - len(vet_x)*sumX2)

  a = np.exp(a)
  b = np.exp(b)

  print(a)
  print(b)

  f_x = lambda x: a * math.pow(b,x)
  #print(f_x(1))

  return f_x

In [ ]:
# a*x^b
def metMMQAXB(vet_x,fx_y):
  sumXY=0; sumX=0; sumY=0; sumX2=0
  for i in range(len(vet_x)):
    sumXY = sumXY + np.log(vet_x[i]) * np.log(fx_y[i])
    sumX = sumX + np.log(vet_x[i])
    sumY = sumY + np.log(fx_y[i])
    sumX2 = sumX2 + math.pow(np.log(vet_x[i]),2)

  b = ((len(vet_x)*sumXY)-sumX*sumY)/(len(vet_x)*sumX2 - math.pow(sumX,2))
  a = ((sumX*sumXY) - sumY*sumX2)/(math.pow(sumX,2) - len(vet_x)*sumX2)

  a = np.exp(a)

  print(a)
  print(b)
  f_x = lambda x: a * math.pow(x,b)

  return f_x

In [ ]:
# a*x^2
def metMMQX2(vet_x,fx_y):
  sumXY=0; sumX=0; sumY=0; sumX2=0; sumX3=0;sumX4=0;sumX2Y=0
  for i in range(len(vet_x)):
    sumXY = sumXY + vet_x[i] * fx_y[i]
    sumX = sumX + vet_x[i]
    sumY = sumY + fx_y[i] 
    sumX2 = sumX2 + math.pow(vet_x[i],2)
    sumX3 = sumX3 + math.pow(vet_x[i],3)
    sumX4 = sumX4 + math.pow(vet_x[i],4)
    sumX2Y = sumX2Y + math.pow(vet_x[i],2)*fx_y[i]

  vet = np.zeros((3, 3))
  vety = np.zeros(3)

  vet[0,0] = len(vet_x)
  vet[0,1] = sumX
  vet[0,2] = sumX2
  vet[1,0] = sumX
  vet[1,1] = sumX2
  vet[1,2] = sumX3
  vet[2,0] = sumX2
  vet[2,1] = sumX3
  vet[2,2] = sumX4
  vety[0] = sumY
  vety[1] = sumXY
  vety[2] = sumX2Y
  
  n = np.linalg.solve(vet,vety)
  f_x = lambda x: n[2]*x**2 + n[1]*x + n[0]

  print(n)
  return f_x

In [ ]:
metNewton ([0.25,0.75,1.25,1.5,2.0],[-0.45,-0.6,0.79,1.88,6.0],4,1.15, 5)

In [ ]:
# LETRA A
X1 = [0, 8, 16, 24, 32, 40]
Y1 =  [14.621, 11.843, 9.870, 8.418, 7.305, 6.413]

In [ ]:
# GRAU 1 ATÉ GRAU 5
i = 0
vet = np.zeros(7)

while i < 5:
  vet[i] = metNewton(X1,Y1,i+1,27,5)
  i += 1

In [ ]:
# LETRA A (estimar 27)
X2 = [0,8,16,24,32,40]
Y2 = [14.621, 11.843, 9.870, 8.418, 7.305, 6.413]

metSplineCub(X2, Y2, 27)

In [ ]:
# VERIFICAR ERRO PARA SLINE
ErroSp = abs(7.986 - 7.96571)
print("Erro de S1:", ErroSp)

# VERIFICAR ERRO PARA GRAU 1 AO 5
for i in range(len(vet)-1):
  ErroNw = abs(7.986 - vet[i])
  print("\nErro de P", i+1, ":", round(ErroNw,5))

In [ ]:
# LETRA A - DIAGRAMA DE DISPERSÃO DOS DADOS PENSAR X FREAR
x = np.array([5.6, 8.5, 11.1, 14.5, 16.7, 22.4], float)
y = np.array([5.0, 12.3, 21.0, 32.9, 47.6,84.7], float)

# LER DADOS
plt.scatter(x, y)

# equação do melhor ajuste
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x, p(x), "r--")

# LEGENDA DOS EIXOS
plt.xlabel("Pensar")
plt.ylabel("Frear")

# PLOTA GRÁFICO
plt.show()

# PRINT RESULT
print("\n\nEquação do melhor ajuste: y = %.4fx + %.4f" % (z[0], z[1]))

In [ ]:
# LETRA A - DIAGRAMA DE DISPERSÃO DOS DADOS
X3 = np.array([0, 4, 8, 12, 16, 20])
Y3 = np.array([67.38, 74.67, 82.74, 91.69, 101.60, 112.58], float)

# LER DADOS
plt.scatter(x, y)

# equação do melhor ajuste
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x, p(x), "r--")

# LEGENDA DOS EIXOS
plt.xlabel("Dia - número de dias")
plt.ylabel("Quantidade x 10^6 - crescimento bacteriológico")

# PLOTA GRÁFICO
plt.show()

# PRINT RESULT
print("\n\nEquação do melhor ajuste: y = %.4fx + %.4f" % (z[0], z[1]))

In [ ]:
# LETRA C - AJUSTE OS DADOS USANDO O MÉTODO DOS MÍNIMOS QUADRADOS
AXB = np.zeros(len(X3))

for i in range(len(X3)):
  AXB[i] = metMMQ(X3,Y3)(X3[i])

plt.scatter(X3,AXB)
#plt.scatter(X3,Y3)
plt.show()


In [ ]:
AX2 = np.zeros(len(X3))
f_x = metMMQX2(X3,Y3)
for i in range(len(X3)):
  AXB[i] = f_x(X3[i])

plt.scatter(X3, AX2)
plt.show()

In [ ]:
# LETRA D, após 30 dias
# Realiza a regressão polinomial
coeficientes = np.polyfit(x, y, 2)
p = np.poly1d(coeficientes)

# Calcula a quantidade de bactérias após 30 dias
x_novo = 30
y_novo = p(x_novo)

print("A quantidade de bactérias após 30 dias é de", round(y_novo, 2), "x 10^6")


In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import math

# Dados de entrada
x = np.array([1, 2, 3, 4, 5])
y = np.array([0.5, 2, 2.9, 3.5, 4])

# Define a equação
def eq(x, a, b):
    return a * np.log(x) + b

# Realiza a regressão não linear
popt, pcov = curve_fit(eq, x, y)

# Obtém os parâmetros a e b
a = popt[0]
b = popt[1]

# Faz a previsão em x = 2.6
x_novo = 2.6
y_novo = a * math.log(x_novo) + b

print("a =", a)
print("b =", b)
print("A previsão de y em x = 2.6 é de", round(y_novo, 2))
